In [1]:
import functools
import random
import textwrap

from huggingface_hub import InferenceApi

In [2]:
class CompletionError(Exception):
    """An error in attempting to complete text with BLOOM."""
    pass

In [3]:
class UnexpectedResponseError(AssertionError):
    """An unexpected error completing text. Probably a bug."""
    pass

In [4]:
class Supplier:
    """As the value of a BLOOM parameter, is called instead of used."""

    __slots__ = ('_func',)

    __match_args__ = ('_func',)

    def __init__(self, func):
        self._func = func
    
    def __repr__(self):
        return f'{type(self).__name__}({self._func!r})'
    
    def __call__(self):
        return self._func()

In [5]:
@functools.cache
def _get_inference_api():
    with open('.hf_token', encoding='utf-8') as file:
        api_token = file.read().strip()

    return InferenceApi(repo_id='bigscience/bloom', token=api_token)

In [6]:
class Completer:
    """High-level interface to Bloom completion."""

    __slots__ = ('_inference', '_text', '__dict__')

    def __init__(self, prompt, **parameters):
        """Create a completer for the given prompt and optional parameters."""
        self._inference = _get_inference_api()
        self.text = prompt

        if any(name.startswith('_') for name in parameters):
            raise TypeError('cannot set parameter name with leading "_"')
        
        self._set_defaults()
        self.__dict__.update(parameters)

    def __str__(self):
        """Prettified version of the text stored so far."""
        raw = self.text.strip().split('\n')
        pretty = ('\n'.join(textwrap.wrap(graf)) for graf in raw)
        return '\n\n'.join(pretty)
    
    @property
    def text(self):
        """The prompt plus any appended completions."""
        return self._text
    
    @text.setter
    def text(self, value):
        if not isinstance(value, str):
            raise TypeError('prompt must be a string')
        if not value:
            raise ValueError('prompt must be nonempty')
        self._text = value

    def __call__(self):
        """Attempt to complete the stored text, and print all text so far."""
        self.complete()
        print(self)
    
    def complete(self):
        """Attempt to complete the stored text, but do not print anything."""
        match self._inference(inputs=self.text, params=self._build_params()):
            case [{'generated_text': completion}]:
                self._text = completion  # Whatever BLOOM returns, we accept.
            case {'error': [*errors]}:
                raise CompletionError(*errors)
            case other_response:
                raise UnexpectedResponseError(other_response)
    
    def _set_defaults(self):
        self.do_sample = True
        self.max_new_tokens = 250
        self.seed = Supplier(random.random)
        self.temperature = 0.75
    
    def _build_params(self):
        return {name: (value() if isinstance(value, Supplier) else value)
                for name, value in sorted(self.__dict__.items())}


In [37]:
bloom1 = Completer("""
The color enforcers came around again. They ripped the cheese off our freshly
baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We license the
color red to you, but you haven't been paying the license fees." I quaked in
fear. My grandfather had gotten both his knees busted by color enforcers.
""".strip().replace('\n', ' ').replace('  ', '\n'))

In [38]:
bloom1()

The color enforcers came around again. They ripped the cheese off our
freshly baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We
license the color red to you, but you haven't been paying the license
fees." I quaked in fear. My grandfather had gotten both his knees
busted by color enforcers. I made a mental note to never use red sauce
again.

I tried to convince him he was wrong. I told him we were licensed to
use red and black. He looked at the paperwork, then looked back at me.

I was getting really worried, but then the henchman got busy with
another customer. I grabbed the paperwork and looked at it. It was
dated 1993, but was valid for all time. I felt better.

I went to the back of the store and asked the cook to make us some
yellow sauce. The cook asked what we wanted it for. I said "blue
cheese."

I thought of my family in Munchkinland. Potatoes are our staple food.
But our potatoes had red and black see

In [48]:
bloom2 = Completer("""
Beverly understood how all the women in her family had fallen in love with this
energy ghost who lived in a lamp and fed parasitically off their vital energy
while making every carnal dream come to life. As a spacefaring physician
""".strip().replace('\n', ' '))

In [50]:
bloom2()

Beverly understood how all the women in her family had fallen in love
with this energy ghost who lived in a lamp and fed parasitically off
their vital energy while making every carnal dream come to life. As a
spacefaring physician who had been unknowingly studying the effects of
biologically compatible energy for years, Beverly knew exactly how to
harness this power for herself. She had never wanted to be like her
mother – a hopeless romantic. But she had wanted to be like her father
– a man who knew how to get what he wanted. She would give this energy
ghost whatever he wanted, and she would get what she needed. At that
moment, Beverly decided that she was done with men.

Beverly continued her research of the energy ghost. She trained to
become an energy healer, and eventually became the best in the
business. She learned to draw upon the power of the energy ghost to
heal herself and to perform some of the most amazing feats of mental
and physical prowess. Beverly was invincible. She w